In [1]:
import pickle
import lightgbm as lgb
import pandas as pd
from sklearn.utils import shuffle

def zuhe(line):
    line['hour'] = line['hour'].astype(str)
    line['weekday'] = line['weekday'].astype(str)
    line['s_ij'] = line['s_ij'].astype(str)
    line['e_ij'] = line['e_ij'].astype(str)

#     line['is_peek'] = line['is_peek'].astype(str)
    line['is_crowd'] = line['is_crowd'].astype(str)
                                   
    line['hour_weekday'] = line['hour'] + line['weekday']
    line['hour_crowd'] = line['hour'] + line['is_crowd']
    line['hour_s'] = line['hour'] + line['s_ij']
    line['hour_e'] = line['hour'] + line['e_ij']

    line['hour'] = line['hour'].astype(int)
#     line['is_peek'] = line['is_peek'].astype(int)
    line['is_crowd'] = line['is_crowd'].astype(int)
    line['weekday'] = line['weekday'].astype(int)
    line['s_ij'] = line['s_ij'].astype(int)
    line['e_ij'] = line['e_ij'].astype(int)
    
    line['hour_weekday'] = line['hour_weekday'].astype(int)
    line['hour_crowd'] = line['hour_crowd'].astype(int)
    line['hour_s'] = line['hour_s'].astype(int)
    line['hour_e'] = line['hour_e'].astype(int)
    return line

with open('./data/train-id4-crowd-grid3.txt', 'rb') as data_file:
    line = pickle.load(data_file)
    line = zuhe(line)#13963746
line = line[(line['Diff_Time']<600)]
line['ID'] = line['ID'].astype('category')
line['s_ij'] = line['s_ij'].astype('category') 
line['e_ij'] = line['e_ij'].astype('category')
train = line
col = [c for c in train if
       c not in ['Unnamed: 0','ID2','s_x', 's_y', 'e_x', 'e_y','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station','O_TIME', 'aver_v','max_v', 'Diff_Time']]
print(col)
X = train[col].values
y = train['Diff_Time'].values
print(X.shape)

# line = line[(line['Diff_Time']<600)]
# line['ID'] = line['ID'].astype('category')
# line['s_ij'] = line['s_ij'].astype('category') 
# line['e_ij'] = line['e_ij'].astype('category') 
# line = shuffle(line)
# train_num = int(0.9 * line.shape[0])
# train = line[:train_num]
# #!
# dev = line[train_num:]
# print(train.shape)
# print(dev.shape)
with open('./data/test-id4-crowd-grid3.txt', 'rb') as data_file:
    test = pickle.load(data_file)
test = zuhe(test)
col1 = [c for c in test if
       c not in ['Unnamed: 0','ID2' , 's_x', 's_y', 'e_x', 'e_y','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                 'Diff_Time','Distance1', 'distance2','TERMINALNO', 'new_dist']]
print(col1)
X_test = test[col1].values
print(X_test.shape)

['ID', 'Distance', 'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'h_aver_diff', 'h_aver_d', 'h_aver_v', 'is_crowd', 's_ij', 'e_ij', 'grid_aver_diff', 'grid_aver_d', 'hour_weekday', 'hour_crowd', 'hour_s', 'hour_e']
(13733553, 23)
['ID', 'Distance', 'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'h_aver_diff', 'h_aver_d', 'h_aver_v', 'is_crowd', 's_ij', 'e_ij', 'grid_aver_diff', 'grid_aver_d', 'hour_weekday', 'hour_crowd', 'hour_s', 'hour_e']
(490380, 23)


In [ ]:
import numpy as np
def train2(X,y,X_test,n_folds):
    from sklearn.model_selection import StratifiedKFold, KFold
    print("folding")
    kf = KFold(n_splits=n_folds,shuffle=True,random_state=2018)
    result = np.zeros((len(X_test), 1))
    print("training")
    count = 0
    for (tr_idx, val_idx) in kf.split(y):
        X_train = X[tr_idx]
        y_train = y[tr_idx]

        X_dev = X[val_idx]
        y_dev = y[val_idx]
        
        print(X_train.shape)
        print(X_dev.shape)
        
        lgb_train  = lgb.Dataset(X_train, y_train,)#feature_name=col,categorical_feature=['ID']
        lgb_eval = lgb.Dataset(X_dev, y_dev,reference=lgb_train)
        params = {'num_leaves':60, 
                  'max_depth':8,
                  'seed':2018,
                  'colsample_bytree':0.8,
                  'subsample':0.9,
                  'num_threads':25,
                  'n_estimators':20000,
                  'learning_rate': 0.1,
                  'objective':'regression_l2',  
                  'metric':'rmse'}
        print(count)
        gbm = lgb.train(params, lgb_train,early_stopping_rounds=200,valid_sets=lgb_eval,verbose_eval=50)

        resultx = gbm.predict(X_test, num_iteration=gbm.best_iteration)
        resultx = np.reshape(resultx,(490380, 1))
        print(resultx.shape,result.shape)
        result = result + resultx
        y_te_pred = gbm.predict(X_dev, num_iteration=gbm.best_iteration)
        #print(log_loss(y_dev, y_te_pred))

        count = count+1
    # 提交结果
    result /= n_folds
    return result

In [ ]:
pred = train2(X,y,X_test,n_folds=10)

folding
training
(12360197, 23)
(1373356, 23)
0


/home/b418-xiwei/anaconda3/envs/dl/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[50]	valid_0's rmse: 39.6765
[100]	valid_0's rmse: 39.0042
[150]	valid_0's rmse: 38.7674
[200]	valid_0's rmse: 38.5938
[250]	valid_0's rmse: 38.4758
[300]	valid_0's rmse: 38.378
[350]	valid_0's rmse: 38.2958
[400]	valid_0's rmse: 38.217
[450]	valid_0's rmse: 38.1563
[500]	valid_0's rmse: 38.1058
[550]	valid_0's rmse: 38.0634
[600]	valid_0's rmse: 38.0216
[650]	valid_0's rmse: 37.978
[700]	valid_0's rmse: 37.9297
[750]	valid_0's rmse: 37.8843
[800]	valid_0's rmse: 37.8502
[850]	valid_0's rmse: 37.8088
[900]	valid_0's rmse: 37.7738
[950]	valid_0's rmse: 37.7458
[1000]	valid_0's rmse: 37.7156
[1050]	valid_0's rmse: 37.6832
[1100]	valid_0's rmse: 37.6544
[1150]	valid_0's rmse: 37.6189
[1200]	valid_0's rmse: 37.5936
[1250]	valid_0's rmse: 37.5701
[1300]	valid_0's rmse: 37.5485
[1350]	valid_0's rmse: 37.5277
[1400]	valid_0's rmse: 37.505
[1450]	valid_0's rmse: 37.483
[1500]	valid_0's rmse: 37.4616
[1550]	valid_0's rmse: 37.4388
[

[13150]	valid_0's rmse: 36.1851
[13200]	valid_0's rmse: 36.1837
[13250]	valid_0's rmse: 36.1821
[13300]	valid_0's rmse: 36.1797
[13350]	valid_0's rmse: 36.1778
[13400]	valid_0's rmse: 36.1749
[13450]	valid_0's rmse: 36.1726
[13500]	valid_0's rmse: 36.1696
[13550]	valid_0's rmse: 36.1681
[13600]	valid_0's rmse: 36.1662
[13650]	valid_0's rmse: 36.1642
[13700]	valid_0's rmse: 36.162
[13750]	valid_0's rmse: 36.1599
[13800]	valid_0's rmse: 36.1582
[13850]	valid_0's rmse: 36.1566
[13900]	valid_0's rmse: 36.1543
[13950]	valid_0's rmse: 36.152
[14000]	valid_0's rmse: 36.1498
[14050]	valid_0's rmse: 36.1482
[14100]	valid_0's rmse: 36.147
[14150]	valid_0's rmse: 36.146
[14200]	valid_0's rmse: 36.1444
[14250]	valid_0's rmse: 36.143
[14300]	valid_0's rmse: 36.1413
[14350]	valid_0's rmse: 36.1395
[14400]	valid_0's rmse: 36.1383
[14450]	valid_0's rmse: 36.1365
[14500]	valid_0's rmse: 36.1355
[14550]	valid_0's rmse: 36.1343
[14600]	valid_0's rmse: 36.1326
[14650]	valid_0's rmse: 36.1308
[14700]	valid

[5950]	valid_0's rmse: 36.648
[6000]	valid_0's rmse: 36.6433
[6050]	valid_0's rmse: 36.6391
[6100]	valid_0's rmse: 36.633
[6150]	valid_0's rmse: 36.6282
[6200]	valid_0's rmse: 36.6229
[6250]	valid_0's rmse: 36.6187
[6300]	valid_0's rmse: 36.6153
[6350]	valid_0's rmse: 36.6106
[6400]	valid_0's rmse: 36.6047
[6450]	valid_0's rmse: 36.5997
[6500]	valid_0's rmse: 36.5937
[6550]	valid_0's rmse: 36.5895
[6600]	valid_0's rmse: 36.586
[6650]	valid_0's rmse: 36.5815
[6700]	valid_0's rmse: 36.5758
[6750]	valid_0's rmse: 36.5704
[6800]	valid_0's rmse: 36.5662
[6850]	valid_0's rmse: 36.5619
[6900]	valid_0's rmse: 36.5561
[6950]	valid_0's rmse: 36.5519
[7000]	valid_0's rmse: 36.5482
[7050]	valid_0's rmse: 36.5433
[7100]	valid_0's rmse: 36.5386
[7150]	valid_0's rmse: 36.5348
[7200]	valid_0's rmse: 36.5311
[7250]	valid_0's rmse: 36.527
[7300]	valid_0's rmse: 36.5229
[7350]	valid_0's rmse: 36.5204
[7400]	valid_0's rmse: 36.517
[7450]	valid_0's rmse: 36.5123
[7500]	valid_0's rmse: 36.5087
[7550]	valid_

[18950]	valid_0's rmse: 36.0335
[19000]	valid_0's rmse: 36.0326
[19050]	valid_0's rmse: 36.0318
[19100]	valid_0's rmse: 36.0309
[19150]	valid_0's rmse: 36.0304
[19200]	valid_0's rmse: 36.03
[19250]	valid_0's rmse: 36.0292
[19300]	valid_0's rmse: 36.0284
[19350]	valid_0's rmse: 36.028
[19400]	valid_0's rmse: 36.027
[19450]	valid_0's rmse: 36.0258
[19500]	valid_0's rmse: 36.0254
[19550]	valid_0's rmse: 36.0245
[19600]	valid_0's rmse: 36.0228
[19650]	valid_0's rmse: 36.0219
[19700]	valid_0's rmse: 36.0213
[19750]	valid_0's rmse: 36.0204
[19800]	valid_0's rmse: 36.0195
[19850]	valid_0's rmse: 36.0183
[19900]	valid_0's rmse: 36.0174
[19950]	valid_0's rmse: 36.0167
[20000]	valid_0's rmse: 36.0161
Did not meet early stopping. Best iteration is:
[19984]	valid_0's rmse: 36.0159
(490380, 1) (490380, 1)
(12360197, 23)
(1373356, 23)
2
Training until validation scores don't improve for 200 rounds.
[50]	valid_0's rmse: 39.7885
[100]	valid_0's rmse: 39.1151
[150]	valid_0's rmse: 38.875
[200]	valid_0'

In [ ]:
# col = [c for c in train if
#        c not in ['Unnamed: 0','ID2','s_x', 's_y', 'e_x', 'e_y','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station','O_TIME', 'aver_v','max_v', 'Diff_Time']]
# print(col)
# X = train[col].values
# y = train['Diff_Time'].values
# X_tes = dev[col].values
# y_tes = dev['Diff_Time'].values
lgb_train  = lgb.Dataset(X, y,)#feature_name=col,categorical_feature=['ID']
lgb_eval = lgb.Dataset(X_tes, y_tes,reference=lgb_train)
params = {'num_leaves':60, 
          'max_depth':8,
          'seed':2018,
          'colsample_bytree':0.8,
          'subsample':0.9,
          'num_threads':25,
          'n_estimators':80000,
          'learning_rate': 0.1,
          'objective':'regression_l2',  
          'metric':'rmse'}
gbm = lgb.train(params, lgb_train,early_stopping_rounds=200,valid_sets=lgb_eval)

In [31]:
print('Save model...')
# save model to file
gbm.save_model('model03.txt')
#model15 36.1558
#model20 36.0846
# Distance        258620
# h_aver_v        189218
# h_aver_diff     184791
# h_aver_d        178070
# hour_weekday    176832
# ID              169274
# s_ij            125871
# e_ij            105354
# hour             88581
# weekday          52152
# dts              47665
# dws              43765
# nts              40770
# nws              37191
# peak_crowd       21215
# is_crowd         19576
# is_peek           7452
# is_rain           6003
# is_workday        2678
#model21 36.0824
# Distance        219484
# h_aver_v        165702
# h_aver_diff     162822
# h_aver_d        156632
# ID              142665
# hour_weekday     95680
# s_ij             94832
# hour_s           83001
# e_ij             79578
# hour_e           60238
# weekday          47771
# dts              42139
# dws              39038
# nts              35874
# nws              33154
# hour_crowd       28953
# hour             24221
# is_crowd         18049
# is_peek           6193
# is_rain           5534
# is_workday        2801
# gbm = lgb.Booster(model_file='model20.txt') #y 750 40.13 3702 #525 35 #575 36.56 #600 13 37.06
print('Start predicting...')
# predict
y_pred = gbm.predict(X_tes, num_iteration=gbm.best_iteration)
# eval
from sklearn.metrics import mean_squared_error
print('The rmse of prediction is:', mean_squared_error(y_tes, y_pred) ** 0.5)

Save model...
Start predicting...
The rmse of prediction is: 35.98342374541739


In [6]:
import pandas as pd
feim = pd.Series(gbm.feature_importance(),index=col)
feim = feim.sort_values(ascending=False)
feim

NameError: name 'gbm' is not defined

In [33]:
with open('./data/test-id4-crowd-grid3.txt', 'rb') as data_file:
    test = pickle.load(data_file)
test = zuhe(test)
col1 = [c for c in test if
       c not in ['Unnamed: 0','ID2' , 's_x', 's_y', 'e_x', 'e_y','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                 'Diff_Time','Distance1', 'distance2','TERMINALNO', 'new_dist']]
print(col1)
X_test = test[col1].values
print(X_test.shape)

y_pred1 =  gbm.predict(X_test, num_iteration=gbm.best_iteration)



['ID', 'Distance', 'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'h_aver_diff', 'h_aver_d', 'h_aver_v', 'is_crowd', 's_ij', 'e_ij', 'grid_aver_diff', 'grid_aver_d', 'hour_weekday', 'hour_crowd', 'hour_s', 'hour_e']
(490380, 23)


In [7]:
with open('./data/test-id4-crowd-grid3.txt', 'rb') as data_file:
    test = pickle.load(data_file)
test = zuhe(test)
test = test[['ID', 'new_dist',  'O_LINENO', 'O_UP', 'Source_Station', 'Target_Station',
       'Distance', 'distance2', 'O_TIME', 'hour', 'is_peek', 'weekday',
       'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'max_v',
       'h_aver_diff', 'h_aver_d', 'h_aver_v', 'TERMINALNO', 
       'is_crowd', 's_ij', 'e_ij','grid_aver_diff', 'grid_aver_d', 'hour_weekday', 'hour_crowd', 'hour_s', 'hour_e']]
test.columns=['ID', 'Distance',  'O_LINENO', 'O_UP', 'Source_Station', 'Target_Station',
       'Distance1', 'distance2', 'O_TIME', 'hour', 'is_peek', 'weekday',
       'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'max_v',
       'h_aver_diff', 'h_aver_d', 'h_aver_v', 'TERMINALNO', 
       'is_crowd', 's_ij', 'e_ij','grid_aver_diff', 'grid_aver_d', 'hour_weekday', 'hour_crowd', 'hour_s', 'hour_e']
col1 = [c for c in test if
       c not in ['Unnamed: 0','ID2' , 's_x', 's_y', 'e_x', 'e_y','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                 'Diff_Time','Distance1', 'distance2','TERMINALNO', 'new_dist']]
print(col1)

X_test = test[col1].values
print(X_test.shape)

# y_pred2 =  gbm.predict(X_test, num_iteration=gbm.best_iteration)

test['pred1'] = pred
test['pred2'] = pred

sub1 = test[['O_LINENO','TERMINALNO', 'O_UP','Source_Station','Target_Station','O_TIME','pred1','pred2','Distance','Distance1']]
sub1['O_TIME'] = pd.to_datetime(sub1['O_TIME'],format='%Y-%m-%d %H:%M:%S')
sub1.columns = ['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime','pred1','pred2','Distance','Distance1']
sub1 = sub1.reset_index()
del sub1['index']

sub=pd.read_csv("./toBePredicted_0607_segment.csv", sep=",")
sub['realTime'] = pd.to_datetime(sub['realTime'],format='%Y-%m-%d %H:%M:%S')
sub2 = sub[['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime','distance']]
sub2=pd.merge(sub2,sub1,on=['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime'],how='left')


['ID', 'Distance', 'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'h_aver_diff', 'h_aver_d', 'h_aver_v', 'is_crowd', 's_ij', 'e_ij', 'grid_aver_diff', 'grid_aver_d', 'hour_weekday', 'hour_crowd', 'hour_s', 'hour_e']
(490380, 23)


/home/b418-xiwei/anaconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
sub2['div_dist'] = sub2['Distance'] / sub2['Distance1']
sub2['new_pred'] = sub2['div_dist'] * sub2['pred1']

import math
for i in range(sub2.shape[0]):
    s = sub2.iloc[i]
    if math.isnan(s['div_dist']):
        sub2.loc[i,'new_pred'] = s['pred1']
        
sub2.to_csv('./toBePredicted_0706_result.csv',sep=",",index=False)#0605 29.2467

print('finished')

finished


In [10]:
print((sub2['pred1']-sub2['pred2']).mean())
print((sub2['pred2']-sub2['new_pred']).mean())
print((sub2['pred1']-sub2['new_pred']).mean())

0.0
4.394280129651668
4.394280129651668


In [11]:
sub2['new_pred'].mean()

131.03270381682265